## Setup

In [1]:
# # Make a directory for pretrained models
# !mkdir -p pretrained
# %cd pretrained

# # Download the model from Hugging Face into the directory
# !huggingface-cli download --resume-download \
#   --local-dir-use-symlinks False \
#   --local-dir InternVL3-78B \
#   OpenGVLab/InternVL3-78B


# # Install the packages
# %pip install -r '/content/drive/MyDrive/toy_ds/requirements.txt'

# # 2️⃣ Install bitsandbytes (GPU-quant):
# %pip install bitsandbytes==0.46.0

# # 3️⃣ Install FlashAttention from its prebuilt wheel to skip the slow build
# %pip install flash-attn==2.8.0.post2 --no-build-isolation

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

toy_ds = '/content/drive/My Drive/datasets/InternVL/toy_ds'
MODEL_PATH = '/content/My Drive/InterVL/InternVL3-78B'

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import torch

num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")

Number of GPUs available: 2


## Video

In [ ]:
import os
import math
import torch
import numpy as np
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

# ─── Prompt ─────────────────────────────────────────────────
prompt_string = """
Please write one detailed but concise paragraph describing the following, based on the video frames or images above:
- The overall scene and weather conditions (e.g., rain, puddles).
- Any visible weapons or use of force:
  • Guns (how many and where)
  • Non-lethal force (e.g., hitting with fists, stones, or sticks)
- Vehicles in the scene:
  • Presence of military vehicles (specify type)
  • License plates or markings (quote exact text)
- Human activity:
  • Actions of uniformed personnel (include uniform colors)
  • Actions of protestors
  • Whether anyone appears to be injured, hurt, or lying on the ground
"""
# ─── Model Setup 78B 8 bit quant ─────────────────────────────────────────────────
MODEL_PATH = "pretrained/InternVL3-38B"
model = AutoModel.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,
    #load_in_4bit=True,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True).eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

# ─── Constants ───────────────────────────────────────────────────
INPUT_SIZE = 448
NUM_FRAMES = 32
GEN_CONFIG = dict(max_new_tokens=1024, do_sample=True)
MEAN = (0.485, 0.456, 0.406)
STD = (0.229, 0.224, 0.225)

# ─── Split Model Across GPUs ──────────────────────────────────────────────
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers

    # Distribute layers evenly, but give half of GPU0 to the vision part
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)

    layer_cnt = 0
    for gpu_idx, count in enumerate(num_layers_per_gpu):
        for _ in range(count):
            device_map[f'language_model.model.layers.{layer_cnt}'] = gpu_idx
            layer_cnt += 1

    # Pin all vision & shared embeddings to GPU0
    vision_keys = [
        'vision_model',
        'mlp1',
        'language_model.model.tok_embeddings',
        'language_model.model.embed_tokens',
        'language_model.output',
        'language_model.model.norm',
        'language_model.model.rotary_emb',
        'language_model.lm_head',
        f'language_model.model.layers.{num_layers - 1}'
    ]
    for key in vision_keys:
        device_map[key] = 0

    return device_map

# ─── Transforms ─────────────────────────────────────────────────
def build_transform():
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB')),
        T.Resize((INPUT_SIZE, INPUT_SIZE), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])

# ─── Load and Preprocess Video ──────────────────────────────────
def get_frame_indices(num_frames, total):
    return np.linspace(0, total - 1, num_frames, dtype=int)

def load_video(video_path, num_frames):
    vr = VideoReader(video_path, ctx=cpu(0))
    transform = build_transform()
    indices = get_frame_indices(num_frames, len(vr))
    pixel_values = [transform(Image.fromarray(vr[i].asnumpy())) for i in indices]
    return torch.stack(pixel_values)  # [num_frames, 3, H, W]

# ─── Inference ──────────────────────────────────────────────────
def infer(video_path):
    video_tensor = load_video(video_path, NUM_FRAMES).to(torch.bfloat16).cuda()
    prompt = ''.join([f'Frame{i+1}: <image>\n' for i in range(NUM_FRAMES)])
    prompt += prompt_string

    response, _ = model.chat(
        tokenizer,
        video_tensor,
        prompt,
        GEN_CONFIG,
        history=None,
        return_history=True
    )
    print("User:", prompt)
    print("Assistant:", response)

# ─── Entry ──────────────────────────────────────────────────────
if __name__ == "__main__":
    infer("toy_ds/videos/._TNS_0169_V.mp4")

Loading checkpoint shards: 100%|██████████| 16/16 [00:01<00:00, 10.39it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
[12:54:19] /github/workspace/src/video/video_reader.cc:83: ERROR opening: /content/drive/MyDrive/toy_ds/videos/TNS_0169_V.mp4, No such file or directory


RuntimeError: Error reading /content/drive/MyDrive/toy_ds/videos/TNS_0169_V.mp4...

## Image

In [ ]:
import math
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer, AutoConfig

# ─── Prompt ─────────────────────────────────────────────────
prompt_string = "Provide a detailed description of each image. Describe the foreground and background separately. Mention any people, objects, and actions clearly. What are the people doing? What expressions or activities are visible? What is the setting or context? Is there violence happening?"

# ─── Config ───────────────────────────────────────────────────────────────
MODEL_PATH = "./pretrained/InternVL3-78B"
INPUT_SIZE   = 448
MAX_PATCHES  = 12
GEN_CONFIG   = dict(max_new_tokens=1024, do_sample=True)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

# ─── Split Model Across GPUs ──────────────────────────────────────────────
def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers

    # Distribute layers evenly, but give half of GPU0 to the vision part
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)

    layer_cnt = 0
    for gpu_idx, count in enumerate(num_layers_per_gpu):
        for _ in range(count):
            device_map[f'language_model.model.layers.{layer_cnt}'] = gpu_idx
            layer_cnt += 1

    # Pin all vision & shared embeddings to GPU0
    vision_keys = [
        'vision_model',
        'mlp1',
        'language_model.model.tok_embeddings',
        'language_model.model.embed_tokens',
        'language_model.output',
        'language_model.model.norm',
        'language_model.model.rotary_emb',
        'language_model.lm_head',
        f'language_model.model.layers.{num_layers - 1}'
    ]
    for key in vision_keys:
        device_map[key] = 0

    return device_map

# ─── Image Preprocessing ─────────────────────────────────────────────────
def build_transform():
    return T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((INPUT_SIZE, INPUT_SIZE), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])

def dynamic_preprocess(image, image_size=448, max_num=12):
    orig_w, orig_h = image.size
    aspect = orig_w / orig_h

    # find best grid (i × j) closest to aspect, with i*j ≤ max_num
    best, best_diff = (1,1), float('inf')
    for i in range(1, max_num+1):
        for j in range(1, max_num+1):
            if i*j > max_num: continue
            diff = abs(aspect - (i/j))
            if diff < best_diff:
                best, best_diff = (i,j), diff

    gw, gh = best
    new_w, new_h = image_size * gw, image_size * gh
    image = image.resize((new_w, new_h))

    tiles = []
    for y in range(gh):
        for x in range(gw):
            box = (x*image_size, y*image_size, (x+1)*image_size, (y+1)*image_size)
            tiles.append(image.crop(box))
    return tiles

def load_image(path, image_size=448, max_num=12):
    img = Image.open(path).convert('RGB')
    tiles = dynamic_preprocess(img, image_size=image_size, max_num=max_num)
    tfm = build_transform()
    return torch.stack([tfm(t) for t in tiles])

# ─── Inference ────────────────────────────────────────────────────────────
def infer(image_path):
    device_map = split_model(MODEL_PATH)
    model     = AutoModel.from_pretrained(
                    MODEL_PATH,
                    torch_dtype=torch.bfloat16,
                    low_cpu_mem_usage=True,
                    use_flash_attn=True,
                    trust_remote_code=True,
                    device_map=device_map
                ).eval()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True, use_fast=False)

    pixel_values = load_image(image_path, max_num=MAX_PATCHES)
    pixel_values = pixel_values.to(torch.bfloat16).cuda()

    question = f"<image>\n{prompt_string}"
    response, _ = model.chat(tokenizer, pixel_values, question, GEN_CONFIG, history=None, return_history=True)

    print("User:", question)
    print("Assistant:", response)

if __name__ == "__main__":
    infer("toy_ds/images/TNS_3773_I.jpg")